In [1]:
import tensorflow as tf
print("Available: ", tf.config.list_physical_devices())


Available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [3]:
FILE_PATH = './Dataset/EURUSD/EURUSD_M30_features+label_v.2.1.csv'


In [4]:
# Custom Dataset
class ForexDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.long)


In [5]:
# Preprocess the data
def preprocess_data(file_path):
    import pandas as pd
    df = pd.read_csv(file_path)

    features = df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA200', 'SMA100', 'SMA50', 'SMA13', 'RSI14']].values
    target = df['signal'].values

    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    sequence_length = 10
    X, y = [], []
    for i in range(len(features) - sequence_length):
        X.append(features[i:i + sequence_length])
        y.append(target[i + sequence_length])

    X = np.array(X)
    y = np.array(y)

    return X, y, scaler

X, y, scaler = preprocess_data(FILE_PATH)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
train_dataset = ForexDataset(X_train, y_train)
test_dataset = ForexDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [7]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, seq_length, d_model=128, nhead=4, num_layers=2):
        super(TransformerModel, self).__init__()

        self.positional_encoding = nn.Parameter(torch.zeros(1, seq_length, input_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(seq_length * d_model, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x += self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.fc(x)
        return x


In [8]:
# Model Parameters
input_dim = X_train.shape[2]
seq_length = X_train.shape[1]
num_classes = len(np.unique(y))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_dim=input_dim, num_classes=num_classes, seq_length=seq_length).to(device)


c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [10]:
def train_model(model, dataloader, criterion, optimizer, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


In [11]:
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")


In [12]:
# Train and Evaluate
train_model(model, train_loader, criterion, optimizer, device)
evaluate_model(model, test_loader, device)


AssertionError: was expecting embedding dimension of 128, but got 10

In [13]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Custom Dataset
class ForexDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.targets[idx], dtype=torch.long)

# Preprocess the data
def preprocess_data(file_path):
    import pandas as pd
    df = pd.read_csv(file_path)

    features = df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA200', 'SMA100', 'SMA50', 'SMA13', 'RSI14']].values
    target = df['signal'].values

    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    sequence_length = 10
    X, y = [], []
    for i in range(len(features) - sequence_length):
        X.append(features[i:i + sequence_length])
        y.append(target[i + sequence_length])

    X = np.array(X)
    y = np.array(y)

    return X, y, scaler

X, y, scaler = preprocess_data(FILE_PATH)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = ForexDataset(X_train, y_train)
test_dataset = ForexDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, seq_length, d_model=128, nhead=4, num_layers=2):
        super(TransformerModel, self).__init__()

        self.input_projection = nn.Linear(input_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, seq_length, d_model))

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(seq_length * d_model, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.input_projection(x)
        x += self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = self.fc(x)
        return x

# Model Parameters
input_dim = X_train.shape[2]
seq_length = X_train.shape[1]
num_classes = len(np.unique(y))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_dim=input_dim, num_classes=num_classes, seq_length=seq_length).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
def train_model(model, dataloader, criterion, optimizer, device, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Evaluation Loop
def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

# Train and Evaluate
train_model(model, train_loader, criterion, optimizer, device)
evaluate_model(model, test_loader, device)


c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
